# Statistical Analysis

In this notebook, we perform some some statistical analysis to understand if the endotype classification have any relationship with output variables (for example, hospital_mortality, vfds_bronch, etc.)

## 1. Import necessary libraries, have function definitions

In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import mannwhitneyu, kruskal, fisher_exact, chi2_contingency

from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Define vfds groups
def categorize_vfds_bronch(x):
    if 0 <= x <= 7:
        return 1
    elif 8 <= x <= 14:
        return 2
    elif 15 <= x <= 21:
        return 3
    elif 22 <= x <= 28:
        return 4
    else:
        return None

In [3]:
# Define vfds groups
def categorize_cluster(x):
    if x == 1:
        return 5
    else:
        return x

In [4]:
# Fit a logistic regression model with robust standard errors
def run_logistic_regression(df, formula, cluster_name='cluster', reference_cluster=2):
    
    # Convert cluster column to categorical
    data = df.copy()
    data[cluster_name] = data[cluster_name].astype('category')

    # Modify formula to explicitly set reference category
    formula = formula.replace(cluster_name, f"C({cluster_name}, Treatment({reference_cluster}))")

    # Fit the logistic regression model
    model = smf.glm(formula=formula, 
                    data=data, 
                    family=sm.families.Binomial()).fit(cov_type='HC0')  # HC0 = Robust SEs
    
    print(model.summary())

    # Compute Odds Ratios (exponentiate the coefficients)
    odds_ratios = np.exp(model.params)
    
    print("\nOdds Ratios:")
    print(odds_ratios)

    print("\nP-values:")
    print(model.pvalues)
    
    return model

In [5]:
# Fit a modified Poisson regression model with Huber-White robust standard error.
def run_modified_poisson_regression(df, formula, cluster_name='cluster', reference_cluster=2):
    
    # Convert cluster column to categorical
    data = df.copy()
    data[cluster_name] = data[cluster_name].astype('category')

    # Modify formula to explicitly set reference category
    formula = formula.replace(cluster_name, f"C({cluster_name}, Treatment({reference_cluster}))")

    # Fit the Poisson model with robust standard errors
    model = smf.glm(formula=formula, 
                    data=data, 
                    family=sm.families.Poisson()).fit(cov_type='HC0')
    
    print(model.summary())
    
    # Calculate and print Relative Risks (Exponentiated Coefficients)
    relative_risks = np.exp(model.params)
    print("\nRelative Risks (exponentiated coefficients):")
    print(relative_risks)

    print("\nP-values:")
    print(model.pvalues)

In [6]:
# Create a summary table for selected variables, stratified by a grouping variable.
def create_summary_table(df, group_var='cluster'):

    continuous_vars = ['BMI', 'Age', 'bronch_day_1', 'sofa_icu', 'sofa_b1', 'vfds_bronch']
    categorical_vars = ['Sex', 'Race', 'Ethnicity', 'mortality_28d', 'hospital_mortality', 'icu_mortality']
    
    summary = {'continuous': {}, 'categorical': {}}
    
    # Determine groups
    groups = sorted(df[group_var].dropna().unique())
    n_groups = len(groups)
    
    # 1. Continuous variables
    for var in continuous_vars:
        # Overall median, IQR
        overall_median = df[var].median()
        overall_iqr = (df[var].quantile(0.25), df[var].quantile(0.75))
        
        # Group-specific median, IQR
        group_summary = {}
        group_series = []
        for g in groups:
            grp_vals = df.loc[df[group_var] == g, var].dropna()
            group_series.append(grp_vals)
            
            if len(grp_vals) > 0:
                g_median = grp_vals.median()
                g_iqr = (grp_vals.quantile(0.25), grp_vals.quantile(0.75))
                group_summary[g] = f"{g_median:.1f} ({g_iqr[0]:.1f} to {g_iqr[1]:.1f})"
            else:
                group_summary[g] = "NA (NA to NA)"
        
        # p-value: Mann-Whitney if 2 groups, Kruskal-Wallis if >2
        if n_groups == 2:
            try:
                stat, pval = mannwhitneyu(group_series[0], group_series[1], alternative='two-sided')
            except:
                pval = np.nan
        else:
            try:
                stat, pval = kruskal(*group_series)
            except:
                pval = np.nan
        
        summary['continuous'][var] = {
            'Overall': f"{overall_median:.1f} ({overall_iqr[0]:.1f} to {overall_iqr[1]:.1f})",
            'By Group': group_summary,
            'p-value': pval
        }

    # 2. Categorical variables
    for var in categorical_vars:
        summary['categorical'][var] = {'By Category': {}}
        
        # Identify all categories
        categories = df[var].dropna().unique()
        
        for cat in categories:
            row_dict = {}
            # Overall count/percent for this category
            is_cat = (df[var] == cat)
            cat_count = is_cat.sum()
            total_count = df[var].notna().sum()
            cat_percent = 100.0 * cat_count / total_count if total_count else 0
            row_dict["Overall"] = f"{cat_count} ({cat_percent:.1f}%)"
            
            # Group-specific counts/percents
            cat_counts = []
            not_cat_counts = []
            for g in groups:
                grp_df = df[df[group_var] == g]
                grp_cat_count = (grp_df[var] == cat).sum()
                grp_total = grp_df[var].notna().sum()
                grp_percent = 100.0 * grp_cat_count / grp_total if grp_total else 0
                row_dict[g] = f"{grp_cat_count} ({grp_percent:.1f}%)"
                
                # For building the 2-row x n-groups table
                cat_counts.append(grp_cat_count)
                not_cat_counts.append(grp_total - grp_cat_count)
            
            # p-value for "this category vs. not this category" across the groups
            contingency = np.array([cat_counts, not_cat_counts])
            if n_groups == 2:
                # Fisher's exact for 2 groups
                try:
                    _, p_val = fisher_exact(contingency)
                except:
                    p_val = np.nan
            else:
                # Chi-square for >2 groups
                try:
                    _, p_val, _, _ = chi2_contingency(contingency)
                except:
                    p_val = np.nan
            
            row_dict["p-value"] = p_val
            
            summary['categorical'][var]['By Category'][cat] = row_dict
    
    return summary

## 2. Load the data

First, let us load the raw data

In [7]:
file_path = "../data/raw_data/Daily_merged_2025-02-28(in).csv"
sheet_name = "in"

# read the raw data
df_raw = pd.read_csv(file_path)
df_raw = df_raw[df_raw["cohort"] == "vap"]
df_raw = df_raw[df_raw["repeat"] == 1]
df_raw = df_raw.dropna(subset=['balf_PD-L1_V1_imputed'])
df_raw["subject_id"] = df_raw["subject_id"].astype(str)
df_raw = df_raw.reset_index(drop=True)
print(df_raw.shape)
df_raw.head()

(466, 2675)


,merged_id,subject_id,cohort,repeat,encoded_id,true_admit_date,admit_date_redcap,icu_admit_date_iths,icu_admit_source,icu_admit_type,...,pc_IL-10_proinf_V1_imputed,pc_IL-12p70_proinf_V1_imputed,pc_IL-13_proinf_V1_imputed,pc_IL-1?_proinf_V1_imputed,pc_IL-2_proinf_V1_imputed,pc_IL-4_proinf_V1_imputed,pc_IL-6_proinf_V1_imputed,pc_TNF-?_proinf_V1_imputed,pc_sRAGE_V1_imputed,pc_TNF-RI_V1_imputed
0,0060cb7f038fc2524edc6c5fd51c1311f7dab5fd460126...,3901,vap,1,bdcedd0872be7c0678cfe00884f0ef3b7f833e00937b48...,2021-02-01T00:00:00.000000000,2021-02-01T00:00:00.000000000,2021-02-01T21:39:00.000000000,Emergency department,Surgical,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,010a9d89899aff76eca797c6b0b88baa321a7c465cc5aa...,3695,vap,1,c1738fe9ecd511185d729136745c384958e6e009e47080...,2018-07-02T00:00:00.000000000,2018-07-02T00:00:00.000000000,2018-07-03T23:50:00.000000000,Emergency department,Neuro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0138d106c8ad7c0ae0712e4ea5a8b9d5d1c4410d2db4bc...,4097,vap,1,ab1b10eb5052132cfa1465402af4d3af7fd4f008c1084a...,2023-08-06T00:00:00.000000000,2023-08-06T00:00:00.000000000,2023-08-06T04:48:00.000000000,Emergency department,Surgical,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0196c36dc652445d5d25d38bcbee46b0e2e40d91884fa9...,3738,vap,1,6f5dff78f3c7c63081fcbced005868465e7f1d9aa1f640...,2019-06-22T00:00:00.000000000,2019-06-22T00:00:00.000000000,2019-07-01T23:52:00.000000000,Outside hospital transfer,Surgical,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01cc89f062a95ba91cd58800437cb2ef29b48b867701b6...,3791,vap,1,09b4890ada6e16740d1a18a7ba5578e02efcb5d1821d89...,2020-02-09T00:00:00.000000000,2020-02-09T00:00:00.000000000,2020-02-08T18:06:00.000000000,Emergency department,Neuro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Then, let us load the biomarker data that also has the clusters

In [8]:
# Biomarker data with clusters
file_path = "../data/clean_data/vap_cluster_assignments_k4_lca.csv"
df_biomarker = pd.read_csv(file_path)
df_biomarker["subject_id"] = df_biomarker["subject_id"].astype(str)
df_biomarker.drop(columns=["Unnamed: 0"], inplace=True)
print(df_biomarker.shape)
df_biomarker.head()

(466, 2)


,subject_id,cluster
0,3901,3
1,3695,4
2,4097,4
3,3738,3
4,3791,4


## 3. Participant Characteristics

Let us now see if there are any interesting patterns across patient groups. We will generate a summary table for selected continuous and categorical variables, stratified by 'cluster'.

In [9]:
# create a dataframe to generate the summary table
summary_columns = ['subject_id', 'BMI', 'Age', 'bronch_day_1', 'sofa_icu', 'sofa_b1', 'Sex', 'Race', 
               'Ethnicity', '28d_mortality', 'hospital_mortality', 'icu_mortality', 'vfds_bronch', 'bronch_day_1_indexed_to_intubation']
summary_df = pd.merge(df_raw[summary_columns], df_biomarker, on='subject_id')
summary_df.rename(columns={"28d_mortality": "mortality_28d"}, inplace=True)

In [10]:
summary_table = create_summary_table(summary_df)
pprint(summary_table)

{'categorical': {'Ethnicity': {'By Category': {'Hispanic or Latino': {np.int64(1): '0 '
                                                                                   '(0.0%)',
                                                                      np.int64(2): '1 '
                                                                                   '(1.8%)',
                                                                      np.int64(3): '0 '
                                                                                   '(0.0%)',
                                                                      np.int64(4): '0 '
                                                                                   '(0.0%)',
                                                                      'Overall': '1 '
                                                                                 '(0.2%)',
                                                                      'p-value': np.float64(0.05784

- Age differences are statistically significant (p = 0.0028), with Group 2 being the youngest (38.7 years) and Group 1 the oldest (55.5 years), suggesting that younger patients might have different health characteristics. 
- Ventilator-free days (vfds_bronch) also vary significantly (p = 0.048), with Group 4 having the lowest median (9 days), indicating prolonged respiratory support.
- Baseline SOFA scores differ across groups (p = 0.003), where Group 2 has the highest median (7.0), suggesting more severe illness at admission.
- While hospital mortality trends upward from Group 1 (16.2%) to Group 4 (27.3%), the p-value (0.085) suggests this difference is not strongly statistically significant.
- BMI differences are nearly significant (p = 0.061), with Group 1 having the highest (28.7) and Group 2 the lowest (26.3).
- Ethnicity and race distribution show that most patients are non-Hispanic (85.6%), with Hispanic/Latino representation highest in Group 3 (11.5%).
- Group 4 patients had the worst ventilator-free outcomes, while Groups 1 and 2 had the best. This could mean Group 4 had more severe illness or slower recovery. `vfds_bronch` is also statistically significant.

## 4. Relative Risk

I am going to use Modified Poisson Regression with robust standard errors to estimate Relative Risk (RR). This method allows direct estimation of RR in prospective studies with binary outcomes. For now, I'll only analyze the relationship between endotypes (clusters) and some outcome variables to explore patterns in the data.

Relative Risk compares the actual probability of an event occurring between groups (More intuitive for prospective studies).

Reference: [Modified Poisson Regression](https://pubmed.ncbi.nlm.nih.gov/15033648/)

In [11]:
# create the RR dataframe for the analysis
RR_columns = ["subject_id", "Age", "Sex", "Ethnicity", "28d_mortality", "hospital_mortality", 
              "icu_mortality", "sofa_icu", "vfds_bronch", "bronch_day_1_indexed_to_intubation"]
RR_df = pd.merge(df_raw[RR_columns], df_biomarker, on='subject_id')
RR_df.rename(columns={"28d_mortality": "mortality_28d"}, inplace=True)
RR_df['vfds_bronch_category'] = RR_df['vfds_bronch'].apply(categorize_vfds_bronch)

In [12]:
run_modified_poisson_regression(df=RR_df, formula='hospital_mortality ~ cluster')

                 Generalized Linear Model Regression Results                  
Dep. Variable:     hospital_mortality   No. Observations:                  466
Model:                            GLM   Df Residuals:                      462
Model Family:                 Poisson   Df Model:                            3
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -245.05
Date:                Sun, 09 Mar 2025   Deviance:                       298.10
Time:                        12:01:17   Pearson chi2:                     370.
No. Iterations:                     5   Pseudo R-squ. (CS):            0.01116
Covariance Type:                  HC0                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

- The Intercept coefficient is -1.93, which translates to a Relative Risk (RR) of 0.145. This means that patients in **Cluster 2** has the lowest risk of hospital mortality (around 14.5% of the reference level). The p-value (<0.05) confirms that this is statistically significant.
- **Cluster 1** has nearly the same risk as Cluster 2 (RR = 1.12, p = 0.77), suggesting no meaningful difference.
- **Cluster 3** shows a 53% higher risk (RR = 1.53), but it's not statistically significant (p = 0.24).
- **Cluster 4** shows an 88% higher risk (RR = 1.88), with a p-value of 0.08, meaning there might be a real difference, but it's just above the threshold (p < 0.05).

In [13]:
run_modified_poisson_regression(df=RR_df, formula='hospital_mortality ~ cluster + Sex + Age')

                 Generalized Linear Model Regression Results                  
Dep. Variable:     hospital_mortality   No. Observations:                  466
Model:                            GLM   Df Residuals:                      460
Model Family:                 Poisson   Df Model:                            5
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -233.53
Date:                Sun, 09 Mar 2025   Deviance:                       275.05
Time:                        12:01:18   Pearson chi2:                     361.
No. Iterations:                     5   Pseudo R-squ. (CS):            0.05887
Covariance Type:                  HC0                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

- The Intercept coefficient is -3.23, translating to a Relative Risk (RR) of 0.0396. This means that the baseline risk of hospital mortality in **Cluster 2** (reference group) is very low (~4%). The p-value (<0.05) confirms strong statistical significance.
- **Cluster 1** has nearly the same risk as Cluster 2 (RR = 0.89, p = 0.75), suggesting no meaningful difference.
- **Cluster 3** shows a 24% higher risk (RR = 1.237), but it's not statistically significant (p = 0.547).
- **Cluster 4** has the highest mortality risk (59% higher than Cluster 2 (RR = 1.588)), but it's not statistically significant (p = 0.187).
- Male patients have an RR of 0.87, meaning their mortality risk is 13% lower compared to females. However, the p-value (0.472) is not significant.
- Each additional year of age increases the risk of hospital mortality by ~3% (RR = 1.03). The p-value (<0.001) is highly significant, confirming that age is a strong predictor of hospital mortality.
- The Pseudo R² increased to 0.058, meaning the model explains more variation in mortality risk after adding Sex and Age. The log-likelihood improved from -245.05 to -233.53, indicating a better model fit.

In [14]:
# run_modified_poisson_regression(df=RR_df, formula='hospital_mortality ~ cluster + Sex + Age + bronch_day_1_indexed_to_intubation')

In [15]:
# run_modified_poisson_regression(df=RR_df, formula='vfds_bronch_category ~ cluster + Sex + Age')

In [16]:
# run_modified_poisson_regression(df=RR_df, formula='hospital_mortality ~ cluster + Sex + Age + Ethnicity')

In [17]:
# run_modified_poisson_regression(df=RR_df, formula='sofa_icu ~ cluster + Sex + Age')

In [18]:
# run_modified_poisson_regression(df=RR_df, formula='mortality_28d ~ cluster + Sex + Age')

In [19]:
# run_modified_poisson_regression(df=RR_df, formula='icu_mortality ~ cluster + Sex + Age')

## 5. Log Regression

I am going to use Logistic Regression with robust standard errors to estimate Odds Ratios (OR). For now, I'll analyze the relationship between endotypes (clusters) and some outcome variables to explore patterns in the data.

Odds Ratio compares the odds of an event occurring in one group vs. another. Overestimates risk when the outcome is common.

In [20]:
# create the log regression dataframe for the analysis
logR_columns = ["subject_id", "Age", "Sex", "Ethnicity", "28d_mortality", "hospital_mortality", 
              "icu_mortality", "bronch_day_1_indexed_to_intubation"]
logR_df = pd.merge(df_raw[logR_columns], df_biomarker, on='subject_id')
logR_df.rename(columns={"28d_mortality": "mortality_28d"}, inplace=True)

In [21]:
run_logistic_regression(df=logR_df, formula='hospital_mortality ~ cluster')

                 Generalized Linear Model Regression Results                  
Dep. Variable:     hospital_mortality   No. Observations:                  466
Model:                            GLM   Df Residuals:                      462
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -233.72
Date:                Sun, 09 Mar 2025   Deviance:                       467.44
Time:                        12:01:18   Pearson chi2:                     466.
No. Iterations:                     4   Pseudo R-squ. (CS):            0.01406
Covariance Type:                  HC0                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

- The Intercept coefficient is -1.77, translating to an Odds Ratio (OR) of 0.17. This means that the baseline odds of hospital mortality in **Cluster 2** are very low (~17%). The p-value (<0.05) confirms strong statistical significance.
- **Cluster 1** has nearly the same odds as Cluster 2 (OR = 1.14, p = 0.765), showing no meaningful difference.
- **Cluster 3** also has increased odds (OR = 1.69), but the p-value (0.231) indicates the effect is not statistically significant.
- **Cluster 4** shows the highest odds of hospital mortality (OR = 2.20, p = 0.068), meaning patients in Cluster 4 have more than twice the odds of dying compared to Cluster 2.

In [22]:
run_logistic_regression(df=logR_df, formula='hospital_mortality ~ cluster + Age + Sex')

                 Generalized Linear Model Regression Results                  
Dep. Variable:     hospital_mortality   No. Observations:                  466
Model:                            GLM   Df Residuals:                      460
Model Family:                Binomial   Df Model:                            5
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -218.68
Date:                Sun, 09 Mar 2025   Deviance:                       437.37
Time:                        12:01:18   Pearson chi2:                     458.
No. Iterations:                     5   Pseudo R-squ. (CS):            0.07567
Covariance Type:                  HC0                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

- The Intercept coefficient is -3.45, translating to an Odds Ratio of 0.0316. This means that the baseline odds of hospital mortality in **Cluster 2** are very low (~3.2%). The p-value (<0.05) confirms strong statistical significance.
- **Clusters 1** (0R=0.84,p-value=0.69) and **Cluster 3** (0R=1.29,p-value=0.57) do not show significant differences in hospital mortality odds compared to Cluster 2.
- **Cluster 4** still has the highest odds of hospital mortality (OR = 1.86), meaning its patients have an 86% higher chance of dying compared to Cluster 2, but the p-value (0.174) means this result is not statistically strong.
- Male patients have an OR of 0.81, meaning their mortality odds are 19% lower compared to females. However, p-value = 0.449 is not significant.
- The coefficient for Age is 0.0386, meaning each additional year of age increases the odds of hospital mortality by ~3.9% (OR = 1.039).
The p-value (<0.05) is highly significant, confirming that age is a strong predictor of hospital mortality. Thus, older patients have significantly higher hospital mortality risk.
- The Pseudo R² increased to 0.075, meaning the model explains more variation in hospital mortality after adding Age and Sex. The log-likelihood improved from -233.72 to -218.68, indicating a better model fit.

In [28]:
# run_logistic_regression(df=logR_df, formula='icu_mortality ~ cluster + Sex + Age')

In [31]:
# run_logistic_regression(df=logR_df, formula='mortality_28d ~ cluster + Sex + Age')

In [29]:
# run_logistic_regression(df=logR_df, formula='hospital_mortality ~ cluster + Sex + Age + bronch_day_1_indexed_to_intubation')